This notebook shows subclass-wise classification report of finetuning results generated previously. 

In [1]:
from pathlib import Path
import pickle
from sklearn.metrics import classification_report
import pandas as pd
from rich import print as rprint
from schelp.utils.config import load_config

paths = load_config(dataset_key="init")

In [2]:
# loading saved results from finetuning run. 
# dev_brain-Jul15-16-47 renamed to balance-1.0-finetuned
with open(paths["data"] / "results" / "balance-1.0-finetuned" / "results.pkl", "rb") as f:
    results = pickle.load(f)
print(results.keys())

dict_keys(['predictions', 'labels', 'results', 'id_maps'])


In [3]:
# this should match the results in 04_adapted_annotation_finetuning.ipynb notebook
res = results["results"]
for k, v in res.items():
    print(f"{k}: {v:0.3f}")

test/accuracy: 0.865
test/precision: 0.870
test/recall: 0.864
test/macro_f1: 0.863


In [4]:
y_true = results["labels"]
y_pred = results["predictions"]

id2type = results["id_maps"]
cell_type_list = list(id2type.values())

y_true = [id2type[i] for i in y_true]
y_pred = [id2type[i] for i in y_pred]


In [5]:
# get classification report

report = classification_report(y_true, y_pred, target_names=cell_type_list, output_dict=True)
df = pd.DataFrame(report).transpose()
display(df.tail(3))

# display full dataframe
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

# reorder rows to match the order of celltypes
df = df.reindex(cell_type_list)


# color any values less than 0.5
def color_negative_red(val):
    color = "#ff9999" if val < 0.5 else "black"
    return "color: %s" % color


display(df.round(3).style.map(color_negative_red))

,precision,recall,f1-score,support
accuracy,0.865216,0.865216,0.865216,0.865216
macro avg,0.870040,0.864160,0.862920,27800.000000
weighted avg,0.871483,0.865216,0.864215,27800.000000


,precision,recall,f1-score,support
Astro_1,0.615000,0.854000,0.715000,178.000000
Astro_2,0.755000,0.829000,0.790000,193.000000
Astro_3,0.941000,0.849000,0.892000,205.000000
Astro_4,0.762000,0.952000,0.846000,208.000000
Astro_5,0.873000,0.629000,0.732000,197.000000
Astro_6-SEAAD,0.847000,0.577000,0.686000,201.000000
Chandelier_1,0.960000,0.866000,0.911000,224.000000
Chandelier_2,0.870000,0.962000,0.913000,208.000000
Endo_1,0.930000,0.996000,0.962000,228.000000
Endo_2,0.889000,0.889000,0.889000,190.000000
